In [6]:
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config)) 

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN,LSTM

In [7]:
model = Sequential()

In [8]:
#Embedding Layer
model.add(Embedding(output_dim=32,
                    input_dim=3800,
                    input_length=380))
model.add(Dropout(0.35))

In [9]:
# model.add(SimpleRNN(units=16))
model.add(LSTM(32))
model.add(Dense(units=256,activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(units=1,activation='sigmoid'))

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 380, 32)           121600    
_________________________________________________________________
dropout_2 (Dropout)          (None, 380, 32)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               8448      
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 138,625
Trainable params: 138,625
Non-trainable params: 0
_________________________________________________________________


In [11]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import re,os
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('',text)
def read_files(filetype):
    path = "data/aclImbd"
    file_list = []
    
    positive_path = path + '/' + filetype + '/pos/'
    for f in os.listdir(positive_path):
        file_list+=[positive_path+f]
        
    negative_path = path + '/' + filetype + '/neg/'
    for f in os.listdir(negative_path):
        file_list+=[negative_path+f]
    
    print('read',filetype, 'files:',len(file_list))
    
    all_labels = ([1] * 12500 + [0] * 12500)
    
    all_texts = []
    for fi in file_list:
        with open(fi, encoding = 'utf8') as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]
    return all_labels, all_texts
y_train,train_text=read_files('train')
y_test,test_text=read_files('test')
token = Tokenizer(num_words=3800)
token.fit_on_texts(train_text)
x_train_seq = token.texts_to_sequences(train_text)
x_test_seq = token.texts_to_sequences(test_text)
x_train = sequence.pad_sequences(x_train_seq, maxlen=380)
x_test = sequence.pad_sequences(x_test_seq, maxlen=380)

read train files: 25000
read test files: 25000


In [12]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [13]:
train_history = model.fit(x_train, y_train, batch_size=100,
                         epochs=10,verbose=2,
                         validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 - 69s - loss: 0.5028 - acc: 0.7421 - val_loss: 0.3992 - val_acc: 0.8328
Epoch 2/10
 - 68s - loss: 0.2744 - acc: 0.8884 - val_loss: 0.3416 - val_acc: 0.8552
Epoch 3/10
 - 68s - loss: 0.2347 - acc: 0.9071 - val_loss: 0.4020 - val_acc: 0.8392
Epoch 4/10
 - 67s - loss: 0.2010 - acc: 0.9223 - val_loss: 0.5098 - val_acc: 0.7994
Epoch 5/10
 - 68s - loss: 0.1779 - acc: 0.9322 - val_loss: 0.4593 - val_acc: 0.8332
Epoch 6/10
 - 67s - loss: 0.1588 - acc: 0.9404 - val_loss: 0.3575 - val_acc: 0.8688
Epoch 7/10
 - 67s - loss: 0.1459 - acc: 0.9445 - val_loss: 0.5679 - val_acc: 0.8170
Epoch 8/10
 - 67s - loss: 0.1396 - acc: 0.9469 - val_loss: 0.8449 - val_acc: 0.7068
Epoch 9/10
 - 67s - loss: 0.1297 - acc: 0.9512 - val_loss: 0.5636 - val_acc: 0.8290
Epoch 10/10
 - 67s - loss: 0.1116 - acc: 0.9581 - val_loss: 0.7271 - val_acc: 0.7870


In [14]:
scores = model.evaluate(x_test, y_test, verbose=1)
scores[1]

25000/25000 [==============================] - 71s 3ms/step


0.84884